# Jan 5, 2024

Mike and I had a meeting and he shared his ```R``` code to get irrigated hay areas from full dataset of CENSUS.

The census link is [here](https://www.nass.usda.gov/Publications/AgCensus/2017/index.php).

I am converting it to Python.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
Shannon_data_dir = data_dir_base + "Shannon_Data/"
Min_dir_base = data_dir_base + "Min_Data/"
NASS_dir = data_dir_base + "/NASS_downloads/"
reOrganized_dir = data_dir_base + "reOrganized/"

In [3]:
import sys
sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [4]:
# import gzip

# file_full = NASS_dir + "2017_cdqt_data.txt.gz"
# with gzip.open(file_full,'rt') as f:
#     for line in f:
#         print('got line', line)
        
        
# import gzip
# with gzip.open(file_full, 'rb') as f:
#     file_content = f.read()

In [5]:
file_full = NASS_dir + "2017_cdqt_data.txt.gz"

In [6]:
# deleting [quotechar='"'] does not make any difference
census_df = pd.read_csv(file_full, compression='gzip', header=0, sep='\t', quotechar='"', low_memory=False)
census_df.head(2)

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,CENSUS_COLUMN,SECTOR_DESC,SHORT_DESC,COMMODITY_DESC,AGG_LEVEL_DESC,STATE_FIPS_CODE,STATE_ALPHA,STATE_NAME,COUNTY_CODE,COUNTY_NAME,DOMAINCAT_DESC,VALUE
0,1,1,1,1,ECONOMICS,FARM OPERATIONS - NUMBER OF OPERATIONS,FARM OPERATIONS,NATIONAL,99,US,US TOTAL,NaN,NaN,NaN,"2,042,220"
1,1,1,1,1,ECONOMICS,FARM OPERATIONS - NUMBER OF OPERATIONS,FARM OPERATIONS,STATE,1,AL,ALABAMA,NaN,NaN,NaN,"40,592"


In [7]:
census_df = census_df[census_df.AGG_LEVEL_DESC == "COUNTY"]
census_df.reset_index(drop=True, inplace=True)

print (f"{census_df.shape = }")
census_df.head(2)

census_df.shape = (4022330, 15)


,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,CENSUS_COLUMN,SECTOR_DESC,SHORT_DESC,COMMODITY_DESC,AGG_LEVEL_DESC,STATE_FIPS_CODE,STATE_ALPHA,STATE_NAME,COUNTY_CODE,COUNTY_NAME,DOMAINCAT_DESC,VALUE
0,2,1,1,2,ECONOMICS,FARM OPERATIONS - NUMBER OF OPERATIONS,FARM OPERATIONS,COUNTY,1,AL,ALABAMA,1.0,AUTAUGA,NaN,371
1,2,1,1,2,ECONOMICS,FARM OPERATIONS - NUMBER OF OPERATIONS,FARM OPERATIONS,COUNTY,2,AK,ALASKA,10.0,ALEUTIAN ISLANDS,NaN,46


In [8]:
%%time
# d <- d %>% unite(fips, STATE_FIPS_CODE, COUNTY_CODE, sep="", na.rm=FALSE)

census_df = rc.census_stateCntyAnsi_2_countyFips(df=census_df, 
                                                 state_fip_col="STATE_FIPS_CODE", 
                                                 county_fip_col="COUNTY_CODE")

census_df.head(2)

CPU times: user 1min 51s, sys: 238 ms, total: 1min 51s
Wall time: 1min 52s


,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,CENSUS_COLUMN,SECTOR_DESC,SHORT_DESC,COMMODITY_DESC,AGG_LEVEL_DESC,STATE_FIPS_CODE,STATE_ALPHA,STATE_NAME,COUNTY_CODE,COUNTY_NAME,DOMAINCAT_DESC,VALUE,county_fips
0,2,1,1,2,ECONOMICS,FARM OPERATIONS - NUMBER OF OPERATIONS,FARM OPERATIONS,COUNTY,01,AL,ALABAMA,001,AUTAUGA,NaN,371,01001
1,2,1,1,2,ECONOMICS,FARM OPERATIONS - NUMBER OF OPERATIONS,FARM OPERATIONS,COUNTY,02,AK,ALASKA,010,ALEUTIAN ISLANDS,NaN,46,02010


In [9]:
census_df = census_df[census_df.SECTOR_DESC == "CROPS"]
census_df.dropna(subset=['VALUE'], inplace=True)
census_df.reset_index(drop=True, inplace=True)

In [10]:
census_df_25 = census_df[census_df.CENSUS_TABLE == 25].copy()

# field <- field[field$SHORT_DESC %like% "ACRES HARVESTED",]
census_df_25 = census_df_25[census_df_25.SHORT_DESC.str.contains(pat="ACRES HARVESTED", case=False)].copy()

census_df_25.reset_index(drop=True, inplace=True)

In [11]:
print (f"{census_df.shape = }")
census_df_25 = census_df_25[~(census_df_25.SHORT_DESC.str.contains(pat="SPRING", case=False))].copy()
census_df_25 = census_df_25[~(census_df_25.SHORT_DESC.str.contains(pat="WINTER", case=False))].copy()
census_df_25 = census_df_25[~(census_df_25.SHORT_DESC.str.contains(pat="PIMA", case=False))].copy()
census_df_25 = census_df_25[~(census_df_25.SHORT_DESC.str.contains(pat="UPLAND", case=False))].copy()
census_df_25 = census_df_25[~(census_df_25.SHORT_DESC.str.contains(pat="OIL TYPE", case=False))].copy()
census_df_25 = census_df_25[~(census_df_25.SHORT_DESC.str.contains(pat="NON-OIL TYPE", case=False))].copy()
print (f"{census_df.shape = }")

census_df.shape = (1173717, 16)
census_df.shape = (1173717, 16)


In [24]:
# field <- field %>% filter(str_detect(SHORT_DESC, "IRRIGAT", negate=TRUE))
field_25     = census_df_25[~(census_df_25.SHORT_DESC.str.contains(pat="IRRIGAT", case=False))].copy()
field_25_irr = census_df_25[  census_df_25.SHORT_DESC.str.contains(pat="IRRIGAT", case=False)].copy()

field_25.reset_index(drop=True, inplace=True)
field_25_irr.reset_index(drop=True, inplace=True)

In [25]:
# field <- field %>% filter(str_detect(SHORT_DESC, "IRRIGAT", negate=TRUE))

In [26]:
field_25.head(2)

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,CENSUS_COLUMN,SECTOR_DESC,SHORT_DESC,COMMODITY_DESC,AGG_LEVEL_DESC,STATE_FIPS_CODE,STATE_ALPHA,STATE_NAME,COUNTY_CODE,COUNTY_NAME,DOMAINCAT_DESC,VALUE,county_fips
0,2,25,6,2,CROPS,BARLEY - ACRES HARVESTED,BARLEY,COUNTY,10,DE,DELAWARE,001,KENT,NaN,"8,433",10001
1,2,25,7,2,CROPS,BARLEY - ACRES HARVESTED,BARLEY,COUNTY,10,DE,DELAWARE,003,NEW CASTLE,NaN,"2,173",10003


In [27]:
field_25_irr.rename(columns={"VALUE" : "VALUE_irr"}, inplace=True)
field_25_irr.rename(columns={"SHORT_DESC" : "SHORT_DESC_irr"}, inplace=True)
field_25_irr.head(2)

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,CENSUS_COLUMN,SECTOR_DESC,SHORT_DESC_irr,COMMODITY_DESC,AGG_LEVEL_DESC,STATE_FIPS_CODE,STATE_ALPHA,STATE_NAME,COUNTY_CODE,COUNTY_NAME,DOMAINCAT_DESC,VALUE_irr,county_fips
0,2,25,6,5,CROPS,"BARLEY, IRRIGATED - ACRES HARVESTED",BARLEY,COUNTY,10,DE,DELAWARE,001,KENT,NaN,"1,802",10001
1,2,25,7,5,CROPS,"BARLEY, IRRIGATED - ACRES HARVESTED",BARLEY,COUNTY,10,DE,DELAWARE,003,NEW CASTLE,NaN,(D),10003


In [28]:
# field <- field %>% select(CENSUS_CHAPTER, CENSUS_TABLE, STATE_ALPHA, CENSUS_ROW, fips, SHORT_DESC, VALUE)
# field_i <- field_i %>% select(CENSUS_CHAPTER, CENSUS_TABLE, CENSUS_ROW, fips, SHORT_DESC, VALUE_I)

s_col = ["CENSUS_CHAPTER", "CENSUS_TABLE", "CENSUS_ROW", "county_fips", "SHORT_DESC", "VALUE"]
field_25 = field_25[s_col]

s_col = ["CENSUS_CHAPTER", "CENSUS_TABLE", "CENSUS_ROW", "county_fips", "SHORT_DESC_irr", "VALUE_irr"]
field_25_irr = field_25_irr[s_col]
field_25_irr.head(2)

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,county_fips,SHORT_DESC_irr,VALUE_irr
0,2,25,6,10001,"BARLEY, IRRIGATED - ACRES HARVESTED","1,802"
1,2,25,7,10003,"BARLEY, IRRIGATED - ACRES HARVESTED",(D)


In [29]:
field_25.head(2)

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,county_fips,SHORT_DESC,VALUE
0,2,25,6,10001,BARLEY - ACRES HARVESTED,"8,433"
1,2,25,7,10003,BARLEY - ACRES HARVESTED,"2,173"


In [30]:
field_25_irr_nonIrr = pd.merge(field_25, field_25_irr, 
                               on=list(field_25_irr.columns)[:-2], how="left")

field_25_irr_nonIrr.head(2)

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,county_fips,SHORT_DESC,VALUE,SHORT_DESC_irr,VALUE_irr
0,2,25,6,10001,BARLEY - ACRES HARVESTED,"8,433","BARLEY, IRRIGATED - ACRES HARVESTED","1,802"
1,2,25,7,10003,BARLEY - ACRES HARVESTED,"2,173","BARLEY, IRRIGATED - ACRES HARVESTED",(D)


In [32]:
# field_merge2 <- field_merge %>% filter(str_detect(VALUE, "(D)", negate=TRUE))
field_25_irr_nonIrr = field_25_irr_nonIrr[field_25_irr_nonIrr.VALUE != "(D)"].copy()

In [33]:
field_25_irr_nonIrr["VALUE"]     = field_25_irr_nonIrr["VALUE"].str.replace(",", "")
field_25_irr_nonIrr["VALUE_irr"] = field_25_irr_nonIrr["VALUE_irr"].str.replace(",", "")

In [34]:
field_25_irr_nonIrr.head(2)

,CENSUS_CHAPTER,CENSUS_TABLE,CENSUS_ROW,county_fips,SHORT_DESC,VALUE,SHORT_DESC_irr,VALUE_irr
0,2,25,6,10001,BARLEY - ACRES HARVESTED,8433,"BARLEY, IRRIGATED - ACRES HARVESTED",1802
1,2,25,7,10003,BARLEY - ACRES HARVESTED,2173,"BARLEY, IRRIGATED - ACRES HARVESTED",(D)


In [36]:
field_25_irr_nonIrr["VALUE"] = field_25_irr_nonIrr["VALUE"].astype(float)
field_25_irr_nonIrr["VALUE_irr"] = field_25_irr_nonIrr["VALUE_irr"].astype(float)
field_25_irr_nonIrr.head(2)

ValueError: could not convert string to float: '(D)'

In [ ]:
field_25_irr_nonIrr.rename(columns={"VALUE_irr" : "harvest"}, inplace=True)
field_25_irr_nonIrr.head(2)

In [ ]:
field_25_irr_nonIrr.harvest.unique()

In [85]:
arg1 = "10"
arg1 = int(arg1)
df = pd.DataFrame(index=range(5))
df["arg1"] = arg1
df["exponents"] = 0

for counter in df.index:
    if counter == 0:
        print(
            f"This line will be printed in the file {error_file_directory} which is defined in template.sh!"
        )
        print(f"{counter = :}")
    df.loc[counter, "exponents"] = arg1**counter

This line will be printed in the file /home/h.noorazar/educational/error/indeks.o which is defined in template.sh!
counter = 0


In [86]:
df

,arg1,exponents
0,10,1
1,10,10
2,10,100
3,10,1000
4,10,10000
